In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import *

# 1. 定义方程

In [98]:
def kinetics(x):
    k1,k2,k3,k4,k5,k6,k7,k8 = np.array(x)
    
    eqn1 = k1 + k2 + k3 + k5 + k6 - kC4
    
    eqn2 = k1/kC4*(FC40-FC4) + 1/3*k7*k8*(k1+4/3*k3-k5) / ((k8-kC4)*(k4+k7-kC4)*kC4) \
         * (FC40 - FC4 + kC4/k8*FC40*((FC40/FC4)**(-k8/kC4)-1)) \
         - 1/3*k7*k8*(k1+4/3*k3-k5) / ((k4+k7-k8)*(k4+k7-kC4)*kC4) \
         * (kC4/(k4+k7)*FC40*((FC40/FC4)**(-(k4+k7)/kC4)-1) - \
           kC4/k8*FC40*((FC40/FC4)**(-k8/kC4)-1)) - FC1
        
    eqn3 = FC1 + (2*k2-k1)/kC4*(FC40-FC4) - FC2
    
    eqn4 = (k1+4/3*k3-k5)/(k4+k7-kC4) * (FC4 - FC40*(FC40/FC4)**(-(k4+k7)/kC4)) - FC3
    
    eqn5 = 1/2*k4*(k1+4/3*k3-k5)/(kC4*(k4+k7-kC4)) \
         * (FC40 - FC4 + kC4/(k4+k7)*FC40*((FC40/FC4)**(-(k4+k7)/kC4)-1)) \
         + FC1 - k1/kC4*(FC40-FC4) - FA6
    
    eqn6 = -k5/kC4*(FC4 - FC40) - FA7
    
    eqn7 = -1/2*k6/kC4*(FC4 - FC40) - FA8
    
    eqn8 = 1/3*k7*(k1+4/3*k3-k5)/((k8-kC4)*(k4+k7-kC4)) \
         * (FC4 - FC40*((FC40/FC4)**(-k8/kC4))) \
         + 1/3*k7*(k1+4/3*k3-k5)/((k4+k7-k8)*(k4+k7-kC4)) \
         * FC40 * ((FC40/FC4)**(-(k4+k7)/kC4) - (FC40/FC4)**(-k8/kC4)) - FA9
    
    return (eqn1,eqn2,eqn3,eqn4,eqn5,eqn6,eqn7,eqn8)

# 2. 读取数据

In [99]:
#sample number, starting from 0
sample = 3

#read constants, MATLAB equivalent is "readtable"
constants = pd.read_csv('constants.csv', index_col = 0)
kC4,FC1,FC2,FC3,FC4,FA6,FA7,FA8,FA9,FC40 = constants.iloc[sample,:].values

#read initial guesses, note that initial guess for k8 is unknown thus default to 1e-8
#MATLAB equivalent is "readtable"
init_guess = pd.read_csv('initial_guess.csv', index_col = 0)
#MATLAB equivalent is x = [x, 1e-8]
ki = np.append(init_guess.iloc[sample,:].values, 1e-8) 

# 3. 求解方程

In [100]:
ls = least_squares(kinetics, ki, method = 'dogbox',bounds = ([0]*8,[np.inf]*8),xtol = 1e-12, ftol = 1e-12)  
ks = ls.x 
print(kinetics(ks))
print(ks)

(-7.751231281237668e-10, 5.530298441414061e-15, 1.2420620087993939e-15, 7.690029169005186e-15, 2.2230481344642783e-15, 4.344614945583913e-15, 4.965429109549113e-15, 4.699257476203922e-15)
[3.13297362e-08 1.39961113e-08 3.09148790e-08 1.74472380e-08
 1.02239381e-08 9.66021221e-09 1.07046317e-08 9.86551979e-09]


# 4. 求解所有方程并储存结果

In [102]:
constants = pd.read_csv('constants.csv', index_col = 0)
init_guess = pd.read_csv('initial_guess.csv', index_col = 0)

#loop over all samples
kss = []
for sample in range(6):
    kC4,FC1,FC2,FC3,FC4,FA6,FA7,FA8,FA9,FC40 = constants.iloc[sample,:].values
    ki = np.append(init_guess.iloc[sample,:].values, 1e-8) 
    ls = least_squares(kinetics, ki, method = 'dogbox',bounds = ([0]*8,[np.inf]*8),xtol = 1e-12, ftol = 1e-12)  
    kss.append(ls.x) 

#export results to a csv file
kss = pd.DataFrame(kss)
kss.columns = ['k'+str(x) for x in range(1,9)]
kss.to_csv('results.csv')